In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import datetime


# Configuration file

In [ ]:
# Remove patients with amount of samples outside of these invervals.
MAX_LENGTH_OF_STAY = 60
MIN_LENGTH_OF_STAY = 8

# Remove patients with samples/day outside of these invervals.
HIGH_SAMPLE_FREQUENCY_LIMIT = 24
LOW_SAMPLE_FREQUENCY_LIMIT = 1

SAVE_DATA = True
SAVE_DATA_AS = f'../data/EWS_0122-0423_scrubbed_O{MIN_LENGTH_OF_STAY}U{MAX_LENGTH_OF_STAY}.csv'
SAVE_DATA_AS


## Load dataset

In [ ]:
df_orig = pd.read_csv('../data/EWS_0122-0423_preprocessed.csv').sort_values('PatientID')
df_orig

## Removing duplicated rows, min amount of datapoints, split patients with over 48h between datapoints

In [ ]:
def get_len_mean_var(df, column_name):
    #Group the DataFrame by the column you are interested in and count the rows in each group
    grouped = df.groupby(column_name).size()

    #Calculate the mean and standard deviation of the number of rows
    mean_count = grouped.mean()
    std_count = grouped.std()
    num_patients = len(df[column_name].unique())
    return num_patients, round(mean_count,1), round(std_count, 1)


In [ ]:
df_scrubb = df_orig.drop_duplicates(subset=df_orig.columns.difference(['Index']), inplace=False)
print('Original Data')
print(f'Number of patients in dataset: {len(df_orig.PatientID.unique())}, rows in original dataset: {len(df_orig)}')

# sort the non duplicated data-frame
df_scrubb = df_scrubb.sort_values(['PatientID', 'Timestamp'])
df_scrubb.Index = np.arange(0,len(df_scrubb))
df_scrubb.set_index(np.arange(0,len(df_scrubb)), inplace=True)


stats = get_len_mean_var(df_orig, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation datapoints: {stats[2]}')

print('\nDuplicated rows')
print(f'Number of duplicated rows: {len(df_orig)-len(df_scrubb)}. That is {round(100-len(df_scrubb)/len(df_orig)*100, 2)}% of the dataset rows.')
stats = get_len_mean_var(df_scrubb, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation datapoints: {stats[2]}')

df_orig.sort_values('Gender')
print("Removing duplicated rows, ignoring gender - assume gender don't change during stay")
print(f"Gender distribution before:\n {df_scrubb.drop_duplicates(subset=['PatientID', 'Gender'])['Gender'].value_counts()}")
df_scrubb = df_orig.sort_values('Gender').drop_duplicates(subset=df_orig.columns.difference(['Index', 'Gender']), keep='first', inplace=False).sort_values('Index')
print(f'Number of duplicated rows: {len(df_orig)-len(df_scrubb)}. That is {round(100-len(df_scrubb)/len(df_orig)*100, 2)}% of the dataset rows.')
print(f"Gender distribution after:\n {df_scrubb.drop_duplicates(subset=['PatientID', 'Gender'])['Gender'].value_counts()}")
stats = get_len_mean_var(df_scrubb, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation datapoints: {stats[2]}')
df_orig_no_dup = df_scrubb.copy()

# remove patients with less than MIN_LENGTH_OF_STAY amount of timesteps
val_count = df_scrubb.PatientID.value_counts()
df_scrubb = df_scrubb[df_scrubb.PatientID.isin(val_count[val_count >= MIN_LENGTH_OF_STAY].index)]
print(f"\nRemoving patients with few datapoints (under {MIN_LENGTH_OF_STAY} datapoints). ")
print(f'Number of rows from patients with under {MIN_LENGTH_OF_STAY} datapoints: {sum(val_count[val_count < MIN_LENGTH_OF_STAY])}. That is {round(sum(val_count[val_count < MIN_LENGTH_OF_STAY])/len(df_orig)*100, 2)}% of the dataset rows.')
stats = get_len_mean_var(df_scrubb, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation datapoints: {stats[2]}')

# check for patients with multiple obeservations on the same timestep
double_data = []
for patID in df_scrubb.PatientID.unique():
    df_patID = df_scrubb[df_scrubb.PatientID==patID].copy()
    if len(df_patID) != len(df_patID.Timestamp.unique()):
        double_data.append(df_patID.PatientID.iloc[0])
to_remove = df_scrubb[df_scrubb.PatientID.isin(double_data)]
# checked manually that all duplicates manually, first notation had 1 value that differed, looking like a typo. 
# removing first of duplicated values.
to_remove = to_remove[to_remove.duplicated(subset=['Timestamp'], keep='first')]
df_scrubb.drop(to_remove.Index, inplace=True)
print(f'Found {len(to_remove)} paris of with the same patient ID and timestamp, removing outlier from each pair from the dataset.')
 ### Splitting patients with mutiple visits and checking for constant gender.
# split patients that have multiple visits
min_hours_to_split = 48
# df = df[df.Index<2000]
i=0
j=0
result = pd.DataFrame()

# set all None like value to 'None' for Gender feature
df_scrubb.Gender = df_scrubb.Gender.astype('str')
df_scrubb.loc[df_scrubb.Gender=='nan', 'Gender'] = 'None'
df_scrubb.loc[df_scrubb.Gender=='U', 'Gender'] = 'None'

print(f"Gender distribution before:\n {df_scrubb.drop_duplicates(subset=['PatientID', 'Gender'])['Gender'].value_counts()}")
print(f'\nLooping through {len(df_scrubb.PatientID.unique())} patients...')
print(f"Splitting patients with more than {min_hours_to_split} hours between data points.")
for patID in tqdm(df_scrubb.PatientID.unique()):
    df_patID = df_scrubb[df_scrubb.PatientID==patID].copy()
    
    # checking for change in gender during the stay, set genter to the value that is not None
    if len(df_scrubb[df_scrubb.PatientID==patID].Gender.unique())>1:
        mask = df_scrubb[df_scrubb.PatientID==patID].Gender.unique() != 'None'
        df_scrubb.loc[df_scrubb.PatientID==patID,'Gender'] =df_scrubb[df_scrubb.PatientID==patID].Gender.unique()[mask][0] 

    # checking for patients with split in datapoints time.
    time_in = min(df_patID.Timestamp)
    time_in_care = (df_patID['Timestamp'] - time_in) / (1000 * 3600)
    df_patID[df_patID.columns[df_patID.columns.get_loc('Timestamp')]] = time_in_care
    df_patID['Difference'] = df_patID['Timestamp'].diff()

    df_scrubb.loc[df_patID.Index, 'Time_in_care'] = time_in_care
    
    if any(df_patID[df_patID['Difference'].notna()]['Difference'].abs() > min_hours_to_split):
        i+=1
        pat_to_split = df_patID[df_patID['Difference'].abs() > min_hours_to_split]
        for k in range(len(pat_to_split)):
            j+=1
            # split patients that has long gaps between datapoints, PatientID + 10 000 000.
            df_scrubb.loc[(df_scrubb.PatientID == pat_to_split.PatientID.iloc[k]) & (df_scrubb.Index < pat_to_split.Index.iloc[k]), 'PatientID'] += 10000000*(k+1)

        result = pd.concat([result, pat_to_split])
print(f"Gender distribution after:\n {df_scrubb.drop_duplicates(subset=['PatientID', 'Gender'])['Gender'].value_counts()}")


val_count = df_scrubb.PatientID.value_counts()
print(f'\nFound {i} amount of patients with gap longer than {min_hours_to_split} hours in their stay. Splitting them into different PatientID, {j} new patients.')
stats = get_len_mean_var(df_scrubb, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation  datapoints: {stats[2]}')

print(f"\nFound {len(val_count[val_count < MIN_LENGTH_OF_STAY])} amount of short stays (under {MIN_LENGTH_OF_STAY} datapoints) after splitting, removing them. " 
      f"That is {round(sum(val_count[val_count < MIN_LENGTH_OF_STAY])/len(df_orig)*100, 2)}% of the dataset rows.")
df_scrubb = df_scrubb[df_scrubb.PatientID.isin(val_count[val_count >= MIN_LENGTH_OF_STAY].index)] # remove splits with less than MIN_LENGTH_OF_STAY datapoints. 
stats = get_len_mean_var(df_scrubb, "PatientID")
print(f'Stats for of dataset, Patients: {stats[0]}, Mean datapoints: {stats[1]} and Standard deviation  datapoints: {stats[2]}')


### Change ward ID to combinde wards with similar patients.


In [ ]:
# Mapping based on info from Ståle Nymo: (NB: TOOK INTO CONSIDERATION ADDED INFO FROM MAIL
# 1: Psychiatry: 106, 105, 107, 70, 72
# 2: Medicine: 84, 83, 110, 64, 65, 85
# 3: Surgical AND Ortopedics (75): 67, 91, 114, 92, 75
# 5: Neurology: 79
# 6: ICU: 96, 82
# 7: Obst / Gyn: 97, 117
# 8: Observation unit: 108
# 9: Unknown

def getWard(w):
    if w == 106 or w == 105 or w == 107 or w == 72 or w == 70:
        return 1
    elif w == 83 or w == 84 or w == 85 or w == 64 or w == 65 or w == 110:
        return 2
    elif w == 67 or w == 91 or w == 92 or w == 114 or w == 75:
        return 3
    elif w == 79:
        return 5
    elif w == 96 or w == 82:
        return 6
    elif w == 97 or w == 117:
       return 7
    elif w == 108:
        return 8
    else:
        return 9

df_scrubb['WardID'] = df_scrubb['WardID'].apply(getWard)
print('Unique Ward ID and counts', np.unique(df_scrubb.WardID, return_counts=True))
df_scrubb

# Stats for original Data-set

In [ ]:
pat, count = np.unique(df_orig_no_dup.PatientID, return_counts=True)

print(f'Amount of patients: {len(np.unique(df_orig_no_dup.PatientID))}')
print(f'Amount less or equal than {MIN_LENGTH_OF_STAY}: {len(count[count < MIN_LENGTH_OF_STAY])}')
print(f'Amount more than {MIN_LENGTH_OF_STAY}: {len(count[count >= MIN_LENGTH_OF_STAY])}')
print(f'Amount more than {MAX_LENGTH_OF_STAY}: {len(count[count >= MAX_LENGTH_OF_STAY])}')

# Plotting histogram
plt.hist(x=count[(count <= MAX_LENGTH_OF_STAY) & (count >= MIN_LENGTH_OF_STAY)], bins=int((MAX_LENGTH_OF_STAY-MIN_LENGTH_OF_STAY) / 3), color='blue', label=f'{MIN_LENGTH_OF_STAY} or more data samples')
plt.hist(x=count[count < MIN_LENGTH_OF_STAY], bins=int((MIN_LENGTH_OF_STAY) / 3), color='orange', label=f'Under {MIN_LENGTH_OF_STAY} data samples')
plt.hist(x=count[count < 4], bins=int((4) / 3), color='red', label=f'Under {4} data samples')
plt.xlabel('Data samples for patients', fontsize=18, weight='bold')
plt.ylabel('Amount of patients', fontsize=18, weight='bold')
plt.title('Patient data samples', fontsize=32, weight='bold')
plt.rc('xtick', labelsize=26) 
plt.rc('ytick', labelsize=12) 
plt.legend(fontsize=18)
plt.show()


### Long staying patients, check data collection interval.

In [ ]:
pat, count = np.unique(df_scrubb.PatientID, return_counts=True)
print(f'Amount of patients with more than {MAX_LENGTH_OF_STAY} datapoints in total: {len(count[count > MAX_LENGTH_OF_STAY])}')

df_long_stay = df_scrubb[np.isin(df_scrubb.PatientID, pat[count>MAX_LENGTH_OF_STAY])]


for i, patID in enumerate(df_long_stay.PatientID.unique()):
    df = df_long_stay[df_long_stay.PatientID==patID].copy()
    time_in = min(df.Timestamp)
    df.Timestamp = (df.Timestamp-time_in) /1000/3600/24
    num_data = len(df)

    y = np.ones(num_data)*i
    plt.scatter(df.Timestamp, y, s=2)

plt.xlabel('Days', fontsize=18)
plt.ylabel('Patient', fontsize=18)
plt.title('Timestamps for patients')
plt.show()

### check sample frequency

In [ ]:
sample_frequency_list = []
low_freq_pat_list = []
high_freq_pat_list = []
no_time_patID = []


print(f'looping through {len(df_scrubb)} rows...')
for i, patID in tqdm(enumerate(df_scrubb.PatientID.unique())):
    df = df_scrubb[df_scrubb.PatientID==patID].copy()
    time_in = min(df.Timestamp)
    time_out = max(df.Timestamp)
    time_spent = (time_out - time_in )/ (1000 * 3600 * 24)
    if time_spent==0:
        no_time_patID.append(patID)
        continue  

    sample_frequency = len(df)/ time_spent

    if sample_frequency>HIGH_SAMPLE_FREQUENCY_LIMIT:
        high_freq_pat_list.append((patID, sample_frequency))
    elif sample_frequency<LOW_SAMPLE_FREQUENCY_LIMIT:
        low_freq_pat_list.append((patID, sample_frequency))
    
    sample_frequency_list.append(sample_frequency)


In [ ]:
print(f'Number of patients checked: {len(df_scrubb.PatientID.unique())}')
print(f'Number of patients with no change in Timestamp: {len(no_time_patID)}')
print(f'Number of patients with sample frequency over {HIGH_SAMPLE_FREQUENCY_LIMIT}: {len(high_freq_pat_list)}')
print(f'Number of patients with sample frequency under {LOW_SAMPLE_FREQUENCY_LIMIT}: {len(low_freq_pat_list)}')

_ = plt.hist(np.round(sample_frequency_list, 0), bins=50, log=True, label='Number of Patients')
# Add vertical line
vline_value = 4  # Value at which the vertical line should be placed
plt.axvline(x=vline_value, color='red', linestyle='--', label='Optimal value')

# Add labels and title
plt.xlabel('Frequency', fontsize=18, weight='bold')
plt.ylabel('#Patients Log scale', fontsize=18, weight='bold')
plt.title('Sample frequency patients', fontsize=24, weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=12) 
plt.legend(fontsize=16)
plt.show()

_ = plt.hist(np.round(sample_frequency_list, 0), bins=50, log=False, label='Number of Patients')
# Add vertical line
vline_value = 4  # Value at which the vertical line should be placed
plt.axvline(x=vline_value, color='red', linestyle='--', label='Optimal value')

# Add labels and title
plt.xlabel('Frequency', fontsize=18, weight='bold')
plt.ylabel('#Patients', fontsize=18, weight='bold')
plt.title('Sample frequency patients', fontsize=24, weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=12) 
plt.legend(fontsize=16)
plt.show()

# Missing data

In [ ]:
print('original data')
display(np.sum(df_orig[df_orig.isna()], axis=0))
print('scrubbed data')
display(np.sum(df_scrubb[df_scrubb.isna()], axis=0))

# Save data

In [ ]:
if SAVE_DATA:
    print(f"saved the scrubbed data at: {SAVE_DATA_AS}")
    df_scrubb.to_csv(SAVE_DATA_AS, index=False)